In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("KODEX200_monthly_data.csv", encoding="utf-8")

### Index는 datetime 형으로 변환하기

In [3]:
df.index = pd.to_datetime(df["Date"])
df.head()

,Date,Open,High,Low,Close,Volume,Change
Date,,,,,,,
2010-01-29,2010-01-29,19052,19066,18668,18742,2717811,-0.026996
2010-02-26,2010-02-26,18611,18683,18514,18559,974615,0.000431
2010-03-31,2010-03-31,19853,19858,19729,19789,883019,-0.003776
2010-04-30,2010-04-30,20331,20403,20313,20377,758179,0.007814
2010-05-31,2010-05-31,18887,19126,18882,19134,1979678,0.012220


### 함수들

In [4]:
# 평균모멘텀 구하기
def 평균모멘텀구하기(data,month):
    result = 0
    for i in range(1,(month+1),1):
        result = data / data.shift(i) + result
    return result/month

# 평균모멘텀 스코어 구하기
def 평균모멘텀스코어구하기(data,month):
    result = 0
    for i in range(1,(month+1),1):
        result = np.where(data / data.shift(i) > 1,1,0) + result
    score = pd.DataFrame(result/month, index=data.index,columns=["12M_Score"]) 
    return score

In [5]:
### 월별 수익
def 월별수익(data):
    return data/data.shift(1)

### 전략 수익 데이타만들기

In [47]:
#1. 자본금 초기화
balance = 1000000

#2. 12개월 평균모멘텀 스코어
# 벤치마크 KODEX200
kodex200 = df["Close"]; kodex200.rename("KODEX200", inplace=True)
score12 = 평균모멘텀스코어구하기(kodex200,12)
# 평균모멘텀스코어와 종목["Close"] 합침
momentum12 = pd.concat([kodex200, score12], axis=1)

#3. system stop = market score, 이동평균으로 구하기
momentum12["market_score_ma5"] = momentum12["12M_Score"].rolling(window=5).mean()

#4 월별수익
momentum12["월별수익"] = 월별수익(momentum12["KODEX200"])

데이터 = []
for i in range(0,len(momentum12)):
    투자금액 = round(balance * momentum12["market_score_ma5"][i])
    매수가능금액 = round(투자금액 * momentum12["12M_Score"][i])
    구매가능수량 = round(매수가능금액 / momentum12["KODEX200"][i])
    매수금액 = 구매가능수량 * momentum12["KODEX200"][i]
    cash = balance - 매수금액
    # 이번달 투자금액 * 다음달 월별수익
    try:
        매도금액 = 매수금액 * (1 - momentum12["월별수익"][i+1])
    except:
        매도금액 = 매수금액
    매도금액 = 매도금액 - 수수료
    balance = cash + 매도금액
    데이터 = [투자금액,매수가능금액,구매가능수량,매수금액,cash,매도금액,balance]
    print (데이터)

[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, na

In [ ]:
    
#4. market score * 잔고 = 투자금액
momentum12["투자금액"] = round(balance * momentum12["market_score_ma5"])

#5. 투자금액 * 12개월 평균 스코어 = 매수가능금액
momentum12["매수가능금액"] = round(momentum12["투자금액"] * momentum12["12M_Score"])

#6. 매수가능금액 / 현재가(Close) = 구매가능수량
momentum12["구매가능수량"] = round(momentum12["매수가능금액"] / momentum12["KODEX200"])

#7. 구매가능수량 * 현재가(Close) = 매수금액
momentum12["매수금액"] = momentum12["구매가능수량"] * momentum12["KODEX200"]

#8. 잔고 - 매수금액 = 현금
momentum12["cash"] = balance - momentum12["매수금액"]

#9. 월별수익 * 매수금액 = 매도금액
momentum12["매도금액"] = momentum12["매수금액"] * (1-월별수익(momentum12["KODEX200"]).shift(-1))

#10. 수수료
수수료 = 0
momentum12["매도금액"] = momentum12["매도금액"] - 수수료

#11 현금 + 매도금액 = 잔고
momentum12["balance"] = momentum12["cash"] + momentum12["매도금액"]

In [ ]:
momentum12

### 수익률 구하기

In [ ]:
each = buy_money/momentum12["Close"].astype(int)
구매금액 = each * momentum12["Close"]
momentum12["buy"] = 구매금액
momentum12["cash"] = 잔고 - 구매금액
mon = 월별수익(momentum12["Close"])
# 1달전 수익률과 현재 투자 금액을 곱하여준다.
momentum12["earing"] = (buy * mon.shift(-1) - buy)
#(momentum12["earing"] + momentum12["cash"]).cumprod()
momentum12

### 그래프 그리기

In [ ]:
from mpl_toolkits.axes_grid1 import host_subplot
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (14,8)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.grid'] = False 

host = host_subplot(111)
par = host.twinx()

host.set_xlabel("Date")
host.set_ylabel("KODEX200")
par.set_ylabel("Score")

p1, = host.plot(momentum12["Close"], label="KODEX200",color = "R")
#p2, = par.plot(momentum12["score"], label="12Score")
p2, = par.plot(momentum6["score"], label="6Score")

leg = plt.legend()

host.yaxis.get_label().set_color("R")
leg.texts[0].set_color("R")

par.yaxis.get_label().set_color(p2.get_color())
leg.texts[1].set_color(p2.get_color())

plt.show()

### 수익곡선 이평구하기

In [ ]:
# 이평 구하기
ma3 = momentum12["score"].rolling(window=3).mean()
ma4 = momentum12["score"].rolling(window=4).mean()
ma5 = momentum12["score"].rolling(window=5).mean()
ma6 = momentum12["score"].rolling(window=6).mean()

In [ ]:
# 그래프 그리기
from mpl_toolkits.axes_grid1 import host_subplot
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (14,8)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.grid'] = False 

host = host_subplot(111)
par = host.twinx()

host.set_xlabel("Date")
host.set_ylabel("KODEX200")
par.set_ylabel("Score")

p1, = host.plot(momentum12["Close"], label="KODEX200",color = "R")
#p2, = par.plot(momentum12["score"], label="12Score")
p2, = par.plot(ma3, label="MA3")
p2, = par.plot(ma4, label="MA4")
#p2, = par.plot(ma5, label="MA5")
#p2, = par.plot(ma6, label="MA")

leg = plt.legend()

host.yaxis.get_label().set_color("R")
leg.texts[0].set_color("R")

par.yaxis.get_label().set_color(p2.get_color())
leg.texts[1].set_color(p2.get_color())

plt.show()